### MAST30034: Applied Data Science Project 1
---
# Preprocessing Part 2: Aggregating Data by MMWR Week
#### Xavier Travers (1178369)

Aggregate all the data by MMWR week (defined [here](https://ndc.services.cdc.gov/wp-content/uploads/MMWR_Week_overview.pdf)).
This means counting trips to and from each of the boroughs per month.
This is done for each of the taxi types.

In [1]:
# imports used throughout this notebook
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
import os
import sys
import re
from itertools import chain

# add homemade helpers
sys.path.insert(1, '../scripts')
import helpers.aggregation_helpers as ah
import helpers.join_helpers as jh

# for printouts
DEBUGGING = True

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName('MAST30034 XT Project 1')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/11 23:57:32 WARN Utils: Your hostname, Polaris resolves to a loopback address: 127.0.1.1; using 172.20.85.243 instead (on interface eth0)
22/08/11 23:57:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/11 23:57:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/11 23:57:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/11 23:57:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# read in the taxi zones dataset
zones_df = spark.read.csv('../data/raw/tlc_zones/zones.csv',
    header = True)
zones_df.limit(5)

OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
1,0.116357453189,MULTIPOLYGON (((-...,0.0007823067885,Newark Airport,1,EWR
2,0.43346966679,MULTIPOLYGON (((-...,0.00486634037837,Jamaica Bay,2,Queens
3,0.0843411059012,MULTIPOLYGON (((-...,0.000314414156821,Allerton/Pelham G...,3,Bronx
4,0.0435665270921,MULTIPOLYGON (((-...,0.000111871946192,Alphabet City,4,Manhattan
5,0.0921464898574,MULTIPOLYGON (((-...,0.000497957489363,Arden Heights,5,Staten Island


### 1. Aggregating the TLC dataset

In [4]:
# TODO: commenting
TLC_NAMES = ['yellow']

In [5]:
# read in the yellow dataset# TODO: commenting
tlc_df = jh.read_stacked_tlc_df(spark, '../data/curated/tlc/cleaned/yellow')
print(f'{tlc_df.count()} ROWS')
print(tlc_df.limit(5))

FileNotFoundError: [Errno 2] No such file or directory: '../data/curated/tlc/cleaned/yellow'

In [ ]:
TLC_GROUP_COLUMNS = [
    'year',
    'cdc_week',
    'week_index',
    'pu_borough',
    'do_borough',
    'passengers'
]
# TODO: commenting
TLC_AGGREGATE_COLUMNS = {
    '*': ['count'],
    'trip_distance': ['total', 'average'],
}

In [ ]:
tlc_df = ah.group_and_aggregate(tlc_df, TLC_GROUP_COLUMNS, 
    TLC_AGGREGATE_COLUMNS)
# TODO: commenting
# force this into memory 
# otherwise writing parquets results in a java executor out of memory error
tlc_df = spark.createDataFrame(tlc_df.collect())

In [ ]:
tlc_df.sort('week_index').limit(5)
# TODO: commenting

In [ ]:
tlc_df.sort('avg_trip_distance', ascending=False).limit(20)

In [ ]:
tlc_df.write.mode('overwrite').parquet('../data/curated/tlc/aggregated/yellow')
# TODO: commenting

### 2. Aggregating the COVID dataset

In [ ]:
# read in the covid dataset
covid_df = spark.read.parquet('../data/curated/virals/covid/cases-by-day')
covid_df.limit(5)
# TODO: commenting

In [ ]:
COVID_GROUP_COLUMNS = [
    'year',
    'cdc_week',
    'week_index',
    'borough'
]
# TODO: commenting
COVID_AGGREGATE_COLUMNS = {
    'cases': ['total', 'average'],
    'deaths': ['total', 'average'],
    'hospitalised': ['total', 'average'],
}

In [ ]:
covid_df = ah.group_and_aggregate(covid_df, COVID_GROUP_COLUMNS, 
    COVID_AGGREGATE_COLUMNS)

# force this into memory 
# otherwise writing parquets results in a java executor out of memory error
covid_df = spark.createDataFrame(covid_df.collect())
# TODO: commenting

In [ ]:
covid_df.sort('week_index').limit(5)
# TODO: commenting

In [ ]:
# save it
# TODO: commenting
covid_df.write.mode('overwrite').parquet('../data/curated/virals/covid/cases-by-week')

### 3. Aggregating the Flu dataset
*Nothing is done here, since the flu dataset is already grouped by MMWR week.*